# Mini-RAG: Comparación Qdrant vs Weaviate

Este notebook implementa y compara dos sistemas Mini-RAG usando:
- **Qdrant** como base de datos vectorial
- **Weaviate** como base de datos vectorial
- **FLAN-T5-small** como modelo de lenguaje para generación
- **all-MiniLM-L6-v2** para embeddings

## Objetivos
1. Construir Mini-RAG con ambas bases vectoriales
2. Evaluar con métricas P@k, R@k y nDCG@k
3. Comparar rendimiento, latencia y facilidad de uso
4. Implementar filtrado por metadatos y consultas híbridas


In [1]:
# Instalación de dependencias
!pip install -q qdrant-client weaviate-client sentence-transformers transformers accelerate pandas numpy scikit-learn tqdm rich


In [2]:
# Imports necesarios
import time
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Modelos y embeddings
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Bases de datos vectoriales
from qdrant_client import QdrantClient
from qdrant_client.http import models as qdrant_models
import weaviate
from weaviate.classes.config import Configure

# Métricas
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import normalize

# Visualización
from tqdm import tqdm
from rich import print as rprint
from rich.table import Table
from rich.console import Console

console = Console()
print("✅ Dependencias importadas correctamente")


✅ Dependencias importadas correctamente


## 1. Configuración de Modelos


In [3]:
# Configuración de modelos
EMBED_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
GEN_MODEL_NAME = 'google/flan-t5-small'

print('🔄 Cargando modelos (esto puede tardar unos minutos)...')

# Modelo de embeddings
embedder = SentenceTransformer(EMBED_MODEL_NAME)
embed_dim = embedder.get_sentence_embedding_dimension()

# Modelo de generación
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_NAME)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL_NAME)

print(f'✅ Modelos cargados:')
print(f'   - Embeddings: {EMBED_MODEL_NAME} (dim={embed_dim})')
print(f'   - Generación: {GEN_MODEL_NAME}')


🔄 Cargando modelos (esto puede tardar unos minutos)...
✅ Modelos cargados:
   - Embeddings: sentence-transformers/all-MiniLM-L6-v2 (dim=384)
   - Generación: google/flan-t5-small


## 2. Corpus de Documentos (30 documentos con metadatos)


In [4]:
# Corpus de 30 documentos con metadatos completos
docs = [
    {"id": 1, "title": "Inteligencia Artificial en Medicina", "text": "La IA se aplica en diagnóstico por imágenes, análisis de genomas y detección temprana de enfermedades. Los algoritmos de aprendizaje automático pueden identificar patrones en radiografías, resonancias magnéticas y tomografías computarizadas.", "source": "Revista Salud Digital", "date": "2024-06-12", "category": "medicina"},
    {"id": 2, "title": "Historia de la imprenta", "text": "La imprenta de Gutenberg revolucionó la difusión del conocimiento en Europa desde el siglo XV. Esta innovación permitió la producción masiva de libros y democratizó el acceso a la información.", "source": "Enciclopedia Histórica", "date": "2023-11-03", "category": "historia"},
    {"id": 3, "title": "Beneficios del té verde", "text": "Contiene antioxidantes como catequinas que ayudan a reducir el estrés oxidativo y mejorar la salud cardiovascular. Estudios demuestran que el consumo regular puede reducir el riesgo de enfermedades cardíacas.", "source": "Portal Nutrición y Vida", "date": "2025-01-15", "category": "salud"},
    {"id": 4, "title": "Energía solar: ventajas y retos", "text": "Es renovable y limpia, pero depende del clima y requiere almacenamiento eficiente. Los paneles solares han reducido significativamente su costo en la última década.", "source": "Energías del Futuro", "date": "2025-04-20", "category": "energia"},
    {"id": 5, "title": "Resumen de la teoría de la relatividad", "text": "Einstein propuso que el espacio-tiempo es curvo y que la gravedad es una manifestación de esa curvatura. Esta teoría revolucionó nuestra comprensión del universo y llevó a predicciones como la existencia de agujeros negros.", "source": "Física Moderna Hoy", "date": "2024-08-05", "category": "fisica"},
    {"id": 6, "title": "¿Qué es el efecto placebo?", "text": "Una mejora en la salud causada por la creencia del paciente en un tratamiento sin efecto real. Este fenómeno demuestra la poderosa conexión entre mente y cuerpo en los procesos de curación.", "source": "Psicología Clínica", "date": "2023-12-01", "category": "medicina"},
    {"id": 7, "title": "Las capas de la atmósfera", "text": "Troposfera, estratósfera, mesosfera, termosfera y exosfera tienen distintas funciones y altitudes. Cada capa tiene características específicas de temperatura, presión y composición química.", "source": "Clima y Ciencia", "date": "2024-03-30", "category": "ciencia"},
    {"id": 8, "title": "Qué es el blockchain", "text": "Es una base de datos distribuida y segura que registra transacciones de forma inmutable. Esta tecnología es la base de las criptomonedas y tiene aplicaciones en contratos inteligentes y trazabilidad.", "source": "Tech Diario", "date": "2025-06-01", "category": "tecnologia"},
    {"id": 9, "title": "La fotosíntesis explicada", "text": "Proceso por el cual las plantas convierten CO₂ y luz solar en oxígeno y glucosa. Este proceso es fundamental para la vida en la Tierra ya que produce el oxígeno que respiramos.", "source": "Biología Básica", "date": "2023-10-21", "category": "biologia"},
    {"id": 10, "title": "Importancia de la lectura en la infancia", "text": "Fomenta el desarrollo del lenguaje, la empatía y la capacidad de concentración. Los niños que leen regularmente desarrollan mejor vocabulario y habilidades de comprensión.", "source": "Educación Hoy", "date": "2024-05-19", "category": "educacion"},
    {"id": 11, "title": "Qué es el cambio climático", "text": "Se refiere al aumento sostenido de la temperatura global y sus efectos en el planeta. Las actividades humanas, especialmente la quema de combustibles fósiles, son la principal causa.", "source": "Observatorio Climático", "date": "2025-02-14", "category": "ambiente"},
    {"id": 12, "title": "Origen del universo: Big Bang", "text": "El modelo del Big Bang plantea una expansión desde un estado extremadamente denso y caliente. Esta teoría explica la formación de elementos ligeros y la radiación cósmica de fondo.", "source": "Astrofísica para Todos", "date": "2024-07-07", "category": "fisica"},
    {"id": 13, "title": "Cómo funciona un motor eléctrico", "text": "Convierte energía eléctrica en energía mecánica mediante fuerzas electromagnéticas. Los motores eléctricos son más eficientes que los de combustión interna.", "source": "Ingeniería 101", "date": "2024-09-11", "category": "tecnologia"},
    {"id": 14, "title": "El Renacimiento en Europa", "text": "Movimiento cultural que valoró la razón, el arte y la ciencia, con figuras como Da Vinci o Galileo. Este período marcó la transición de la Edad Media a la Edad Moderna.", "source": "Arte e Historia", "date": "2023-04-03", "category": "historia"},
    {"id": 15, "title": "Python vs JavaScript", "text": "Python destaca en ciencia de datos, mientras que JavaScript domina en desarrollo web. Ambos lenguajes tienen ecosistemas ricos y comunidades activas.", "source": "Blog Código Abierto", "date": "2025-08-10", "category": "tecnologia"},
    {"id": 16, "title": "Qué es el CO₂ y por qué importa", "text": "Es un gas de efecto invernadero cuya acumulación intensifica el calentamiento global. La concentración atmosférica de CO₂ ha aumentado significativamente desde la revolución industrial.", "source": "EcoMundo", "date": "2025-03-22", "category": "ambiente"},
    {"id": 17, "title": "Neuroplasticidad: el cerebro que cambia", "text": "El cerebro puede reorganizarse y formar nuevas conexiones en respuesta al aprendizaje. Esta capacidad se mantiene durante toda la vida, aunque disminuye con la edad.", "source": "Revista Mente Abierta", "date": "2024-02-28", "category": "medicina"},
    {"id": 18, "title": "Aportes de Marie Curie", "text": "Pionera en radioactividad, fue la primera persona en recibir dos premios Nobel. Sus investigaciones abrieron el camino para la medicina nuclear y la física atómica.", "source": "Científicas que hicieron historia", "date": "2023-08-10", "category": "historia"},
    {"id": 19, "title": "Cómo usar una brújula", "text": "Una brújula señala el norte magnético y ayuda en la orientación geográfica. Es una herramienta fundamental para la navegación terrestre y marítima.", "source": "Manual de Supervivencia", "date": "2024-01-05", "category": "supervivencia"},
    {"id": 20, "title": "Qué es una VPN y cómo protege tu privacidad", "text": "Una VPN cifra tu conexión, ocultando tu IP y actividad en línea. Es especialmente útil cuando se conecta a redes WiFi públicas.", "source": "Seguridad Digital Hoy", "date": "2025-05-01", "category": "tecnologia"},
    {"id": 21, "title": "Alimentos ricos en fibra", "text": "Legumbres, frutas, cereales integrales y semillas favorecen la digestión y la salud intestinal. Una dieta rica en fibra puede prevenir enfermedades cardiovasculares.", "source": "Guía Nutricional", "date": "2024-11-09", "category": "salud"},
    {"id": 22, "title": "Fases de la Luna", "text": "Incluyen luna nueva, cuarto creciente, luna llena y cuarto menguante, en un ciclo de ~29 días. Estas fases han sido utilizadas por civilizaciones para medir el tiempo.", "source": "Astronomía Básica", "date": "2023-09-12", "category": "ciencia"},
    {"id": 23, "title": "¿Qué es el machine learning?", "text": "Es una rama de la IA donde los algoritmos aprenden patrones a partir de datos. Se aplica en reconocimiento de imágenes, procesamiento de lenguaje natural y sistemas de recomendación.", "source": "Aprendizaje Automático Hoy", "date": "2025-07-17", "category": "tecnologia"},
    {"id": 24, "title": "El alfabeto griego", "text": "Usado en matemáticas y ciencia, contiene letras como alfa, beta, gamma y delta. Muchas constantes y variables científicas utilizan letras griegas.", "source": "Lenguas Antiguas", "date": "2023-03-30", "category": "educacion"},
    {"id": 25, "title": "Cómo se forma un arcoíris", "text": "La luz solar se refracta y refleja dentro de gotas de agua, separando colores por longitud de onda. Este fenómeno óptico requiere sol y lluvia simultáneamente.", "source": "Física Natural", "date": "2024-04-15", "category": "fisica"},
    {"id": 26, "title": "Importancia de los polinizadores", "text": "Abejas, mariposas y aves ayudan en la reproducción de plantas y producción de alimentos. Sin polinizadores, muchos cultivos no podrían reproducirse.", "source": "Biodiversidad Global", "date": "2025-03-10", "category": "biologia"},
    {"id": 27, "title": "Ventajas del ejercicio regular", "text": "Reduce el riesgo de enfermedades, mejora el estado de ánimo y fortalece el sistema inmune. La OMS recomienda al menos 150 minutos de ejercicio moderado por semana.", "source": "Salud Activa", "date": "2024-12-18", "category": "salud"},
    {"id": 28, "title": "¿Qué es una estrella fugaz?", "text": "Es un meteoro que arde al entrar en la atmósfera terrestre, creando un brillo breve. La mayoría son partículas de polvo cósmico del tamaño de granos de arena.", "source": "Curiosidades del Espacio", "date": "2023-06-06", "category": "ciencia"},
    {"id": 29, "title": "Células madre: usos médicos", "text": "Las células madre pueden regenerar tejidos, tratar enfermedades como leucemia y formar la base de terapias regenerativas. Representan una esperanza para tratar enfermedades degenerativas.", "source": "Medicina del Futuro", "date": "2024-10-02", "category": "medicina"},
    {"id": 30, "title": "¿Qué es la economía circular?", "text": "Es un modelo económico que busca reducir residuos reutilizando, reciclando y prolongando la vida de los productos. Contrasta con el modelo lineal de 'usar y tirar'.", "source": "Economía Sostenible", "date": "2025-08-01", "category": "economia"}
]

print(f"✅ Corpus creado con {len(docs)} documentos")
print(f"📊 Categorías: {len(set(d['category'] for d in docs))}")
print(f"📅 Rango de fechas: {min(d['date'] for d in docs)} - {max(d['date'] for d in docs)}")


✅ Corpus creado con 30 documentos
📊 Categorías: 12
📅 Rango de fechas: 2023-03-30 - 2025-08-10


## 3. Funciones Auxiliares


In [5]:
def embed_texts(texts: List[str]) -> np.ndarray:
    """Genera embeddings para una lista de textos"""
    vecs = embedder.encode(texts, show_progress_bar=False, convert_to_numpy=True)
    return normalize(vecs)

def generate_answer(prompt: str, max_length: int = 128) -> str:
    """Genera respuesta usando FLAN-T5"""
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=512)
    outputs = gen_model.generate(**inputs, max_length=max_length, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def chunk_text(text: str, max_tokens: int = 90) -> List[str]:
    """Divide texto en chunks por palabras"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_tokens):
        chunk = " ".join(words[i:i+max_tokens])
        if chunk.strip():
            chunks.append(chunk)
    return chunks

print("✅ Funciones auxiliares definidas")


✅ Funciones auxiliares definidas


## 4. Preguntas de Prueba con Ground Truth


In [6]:
# 10 preguntas de prueba con documentos relevantes identificados
test_queries = [
    {
        "question": "¿En qué áreas de la medicina se usa la inteligencia artificial?",
        "relevant_docs": [1],  # Inteligencia Artificial en Medicina
        "category": "medicina"
    },
    {
        "question": "¿Cuál es la función principal de la fotosíntesis?",
        "relevant_docs": [9],  # La fotosíntesis explicada
        "category": "biologia"
    },
    {
        "question": "¿Qué propone la teoría del Big Bang sobre el origen del universo?",
        "relevant_docs": [12],  # Origen del universo: Big Bang
        "category": "fisica"
    },
    {
        "question": "¿Cuáles son las capas de la atmósfera terrestre?",
        "relevant_docs": [7],  # Las capas de la atmósfera
        "category": "ciencia"
    },
    {
        "question": "¿Qué es una VPN y cómo protege la privacidad?",
        "relevant_docs": [20],  # Qué es una VPN y cómo protege tu privacidad
        "category": "tecnologia"
    },
    {
        "question": "¿Cuáles son los beneficios del té verde para la salud?",
        "relevant_docs": [3],  # Beneficios del té verde
        "category": "salud"
    },
    {
        "question": "¿Para qué se utilizan las células madre en medicina?",
        "relevant_docs": [29],  # Células madre: usos médicos
        "category": "medicina"
    },
    {
        "question": "¿Qué es el efecto placebo y cómo funciona?",
        "relevant_docs": [6],  # ¿Qué es el efecto placebo?
        "category": "medicina"
    },
    {
        "question": "¿Cómo afecta el CO₂ al cambio climático?",
        "relevant_docs": [16, 11],  # CO₂ y cambio climático
        "category": "ambiente"
    },
    {
        "question": "¿Qué es la economía circular y cómo funciona?",
        "relevant_docs": [30],  # ¿Qué es la economía circular?
        "category": "economia"
    }
]

print(f"✅ {len(test_queries)} preguntas de prueba definidas")
print(f"📝 Categorías cubiertas: {set(q['category'] for q in test_queries)}")


✅ 10 preguntas de prueba definidas
📝 Categorías cubiertas: {'ambiente', 'economia', 'biologia', 'ciencia', 'fisica', 'salud', 'medicina', 'tecnologia'}


## 5. Métricas de Evaluación


In [7]:
def precision_at_k(retrieved_ids: List[int], relevant_ids: List[int], k: int) -> float:
    """Calcula Precision@k"""
    retrieved_k = retrieved_ids[:k]
    relevant_retrieved = sum(1 for doc_id in retrieved_k if doc_id in relevant_ids)
    return relevant_retrieved / k if k > 0 else 0.0

def recall_at_k(retrieved_ids: List[int], relevant_ids: List[int], k: int) -> float:
    """Calcula Recall@k"""
    retrieved_k = retrieved_ids[:k]
    relevant_retrieved = sum(1 for doc_id in retrieved_k if doc_id in relevant_ids)
    return relevant_retrieved / len(relevant_ids) if len(relevant_ids) > 0 else 0.0

def ndcg_at_k(retrieved_ids: List[int], relevant_ids: List[int], k: int) -> float:
    """Calcula nDCG@k usando relevancia binaria"""
    retrieved_k = retrieved_ids[:k]

    # Calcular DCG
    dcg = 0.0
    for i, doc_id in enumerate(retrieved_k):
        if doc_id in relevant_ids:
            dcg += 1.0 / np.log2(i + 2)  # +2 porque log2(1) = 0

    # Calcular IDCG (mejor ordenamiento posible)
    idcg = 0.0
    for i in range(min(len(relevant_ids), k)):
        idcg += 1.0 / np.log2(i + 2)

    return dcg / idcg if idcg > 0 else 0.0

print("✅ Métricas de evaluación implementadas")


✅ Métricas de evaluación implementadas


## 6. Implementación Mini-RAG con Qdrant


In [8]:
# Configurar cliente Qdrant (modo embebido)
DB_PATH = "/tmp/qdrant_db"
qdrant_client = QdrantClient(path=DB_PATH)
QDRANT_COLLECTION = "mini_rag_docs"

# Recrear colección
try:
    qdrant_client.delete_collection(QDRANT_COLLECTION)
except:
    pass

qdrant_client.create_collection(
    collection_name=QDRANT_COLLECTION,
    vectors_config=qdrant_models.VectorParams(
        size=embed_dim,
        distance=qdrant_models.Distance.COSINE
    )
)

print(f"✅ Qdrant configurado: colección '{QDRANT_COLLECTION}' creada")


✅ Qdrant configurado: colección 'mini_rag_docs' creada


In [9]:
# Indexar documentos en Qdrant
points = []
point_id = 1

for doc in tqdm(docs, desc="Indexando en Qdrant"):
    # Crear contenido completo
    full_text = f"{doc['title']}. {doc['text']}"

    # Generar embedding
    embedding = embedder.encode(full_text, normalize_embeddings=True)

    # Crear punto
    point = qdrant_models.PointStruct(
        id=point_id,
        vector=embedding.tolist(),
        payload={
            "doc_id": doc["id"],
            "title": doc["title"],
            "text": doc["text"],
            "source": doc["source"],
            "date": doc["date"],
            "category": doc["category"],
            "full_text": full_text
        }
    )
    points.append(point)
    point_id += 1

# Insertar todos los puntos
qdrant_client.upsert(collection_name=QDRANT_COLLECTION, points=points)

# Verificar inserción
count = qdrant_client.count(collection_name=QDRANT_COLLECTION).count
print(f"✅ {count} documentos indexados en Qdrant")


Indexando en Qdrant: 100%|██████████| 30/30 [00:00<00:00, 47.53it/s]


✅ 30 documentos indexados en Qdrant


## 7. Implementación Mini-RAG con Weaviate (Simulado)


In [10]:
# Implementación simulada de Weaviate para comparación
class MockWeaviate:
    def __init__(self):
        self.vectors = None
        self.documents = []
        self.embeddings = []

    def add_documents(self, docs, embeddings):
        self.documents = docs
        self.embeddings = np.array(embeddings)
        print(f"✅ {len(docs)} documentos indexados en Weaviate (simulado)")

    def search(self, query_vector, k=5, filters=None):
        """Búsqueda por similitud coseno con filtros opcionales"""
        similarities = np.dot(self.embeddings, query_vector.reshape(-1, 1)).flatten()

        # Aplicar filtros si existen
        valid_indices = np.arange(len(self.documents))
        if filters:
            mask = np.ones(len(self.documents), dtype=bool)
            for key, value in filters.items():
                if isinstance(value, tuple) and len(value) == 2:  # Rango de fechas
                    dates = [doc[key] for doc in self.documents]
                    mask &= np.array([(d >= value[0] and d <= value[1]) for d in dates])
                else:  # Igualdad exacta
                    values = [doc.get(key) for doc in self.documents]
                    mask &= np.array([v == value for v in values])
            valid_indices = valid_indices[mask]
            similarities = similarities[mask]

        # Obtener top-k
        if len(similarities) == 0:
            return []

        top_indices = np.argsort(similarities)[::-1][:k]
        results = []
        for idx in top_indices:
            actual_idx = valid_indices[idx] if filters else idx
            results.append({
                'doc_id': self.documents[actual_idx]['id'],
                'score': float(similarities[idx]),
                'title': self.documents[actual_idx]['title'],
                'text': self.documents[actual_idx]['text'],
                'source': self.documents[actual_idx]['source'],
                'category': self.documents[actual_idx]['category']
            })

        return results

# Crear e indexar en Weaviate simulado
mock_weaviate = MockWeaviate()

# Generar embeddings para todos los documentos
weaviate_embeddings = []
for doc in tqdm(docs, desc="Indexando en Weaviate"):
    full_text = f"{doc['title']}. {doc['text']}"
    embedding = embedder.encode(full_text, normalize_embeddings=True)
    weaviate_embeddings.append(embedding)

mock_weaviate.add_documents(docs, weaviate_embeddings)


Indexando en Weaviate: 100%|██████████| 30/30 [00:00<00:00, 32.52it/s]

✅ 30 documentos indexados en Weaviate (simulado)


## 8. Clases RAG para Ambos Sistemas


In [11]:
class QdrantRAG:
    def __init__(self, client, collection_name):
        self.client = client
        self.collection_name = collection_name

    def search(self, query: str, k: int = 5, filters: Optional[Dict] = None) -> Tuple[List[Dict], float]:
        """Búsqueda semántica con filtros opcionales"""
        start_time = time.time()

        # Generar embedding de la consulta
        query_vector = embedder.encode(query, normalize_embeddings=True).tolist()

        # Construir filtros de Qdrant
        qdrant_filter = None
        if filters:
            conditions = []
            for key, value in filters.items():
                if isinstance(value, tuple) and len(value) == 2:  # Rango de fechas
                    # Convert date strings to timestamps for Qdrant filtering
                    try:
                        start_timestamp = datetime.strptime(value[0], '%Y-%m-%d').timestamp()
                        end_timestamp = datetime.strptime(value[1], '%Y-%m-%d').timestamp()
                        conditions.append(
                            qdrant_models.FieldCondition(
                                key=key,
                                range=qdrant_models.Range(gte=start_timestamp, lte=end_timestamp)
                            )
                        )
                    except ValueError:
                        print(f"Error: Invalid date format in filter for key '{key}'. Expected YYYY-MM-DD.")
                        continue
                else:  # Igualdad exacta
                    conditions.append(
                        qdrant_models.FieldCondition(
                            key=key,
                            match=qdrant_models.MatchValue(value=value)
                        )
                    )
            if conditions:
                qdrant_filter = qdrant_models.Filter(must=conditions)

        # Realizar búsqueda
        hits = self.client.query_points(
            collection_name=self.collection_name,
            query=query_vector,
            query_filter=qdrant_filter,
            limit=k,
            with_payload=True
        ).points

        search_time = time.time() - start_time

        results = []
        for hit in hits:
            results.append({
                "doc_id": hit.payload["doc_id"],
                "score": hit.score,
                "title": hit.payload["title"],
                "text": hit.payload["text"],
                "source": hit.payload["source"],
                "category": hit.payload["category"]
            })

        return results, search_time

    def query(self, question: str, k: int = 3, filters: Optional[Dict] = None) -> Dict:
        """Consulta completa RAG: búsqueda + generación"""
        start_time = time.time()

        # Búsqueda
        search_results, search_time = self.search(question, k=k, filters=filters)

        if not search_results:
            return {
                "answer": "No encontré información relevante.",
                "sources": [],
                "search_results": [],
                "search_time": search_time,
                "total_time": time.time() - start_time
            }

        # Construir contexto
        context_parts = []
        for result in search_results:
            context_parts.append(f"Título: {result['title']}\\nTexto: {result['text']}")

        context = "\\n\\n".join(context_parts)

        # Crear prompt
        prompt = f"""Responde la pregunta usando solo la información proporcionada. Sé conciso y preciso.

Información:
{context}

Pregunta: {question}
Respuesta:"""

        # Generar respuesta
        answer = generate_answer(prompt, max_length=150)

        return {
            "answer": answer,
            "sources": list(set([r["source"] for r in search_results])),
            "search_results": search_results,
            "search_time": search_time,
            "total_time": time.time() - start_time
        }

class WeaviateRAG:
    def __init__(self, mock_client):
        self.client = mock_client

    def search(self, query: str, k: int = 5, filters: Optional[Dict] = None) -> Tuple[List[Dict], float]:
        """Búsqueda semántica con filtros opcionales"""
        start_time = time.time()

        # Generar embedding de la consulta
        query_vector = embedder.encode(query, normalize_embeddings=True)

        # Realizar búsqueda
        results = self.client.search(query_vector, k=k, filters=filters)

        search_time = time.time() - start_time
        return results, search_time

    def query(self, question: str, k: int = 3, filters: Optional[Dict] = None) -> Dict:
        """Consulta completa RAG: búsqueda + generación"""
        start_time = time.time()

        # Búsqueda
        search_results, search_time = self.search(question, k=k, filters=filters)

        if not search_results:
            return {
                "answer": "No encontré información relevante.",
                "sources": [],
                "search_results": [],
                "search_time": search_time,
                "total_time": time.time() - start_time
            }

        # Construir contexto
        context_parts = []
        for result in search_results:
            context_parts.append(f"Título: {result['title']}\\nTexto: {result['text']}")

        context = "\\n\\n".join(context_parts)

        # Crear prompt
        prompt = f"""Responde la pregunta usando solo la información proporcionada. Sé conciso y preciso.

Información:
{context}

Pregunta: {question}
Respuesta:"""

        # Generar respuesta
        answer = generate_answer(prompt, max_length=150)

        return {
            "answer": answer,
            "sources": list(set([r["source"] for r in search_results])),
            "search_results": search_results,
            "search_time": search_time,
            "total_time": time.time() - start_time
        }

# Crear instancias RAG
qdrant_rag = QdrantRAG(qdrant_client, QDRANT_COLLECTION)
weaviate_rag = WeaviateRAG(mock_weaviate)

print("✅ Clases RAG implementadas")

✅ Clases RAG implementadas


## 9. Evaluación Completa y Comparación


In [12]:
def evaluate_rag_system(rag_system, system_name: str, test_queries: List[Dict], k: int = 5) -> Dict:
    """Evalúa un sistema RAG con las métricas definidas"""
    results = {
        "system_name": system_name,
        "queries": [],
        "metrics": {
            "precision_at_k": [],
            "recall_at_k": [],
            "ndcg_at_k": [],
            "search_times": [],
            "total_times": []
        }
    }

    for query_data in tqdm(test_queries, desc=f"Evaluando {system_name}"):
        question = query_data["question"]
        relevant_docs = query_data["relevant_docs"]

        # Ejecutar consulta
        response = rag_system.query(question, k=k)

        # Extraer IDs de documentos recuperados
        retrieved_ids = [result["doc_id"] for result in response["search_results"]]

        # Calcular métricas
        p_at_k = precision_at_k(retrieved_ids, relevant_docs, k)
        r_at_k = recall_at_k(retrieved_ids, relevant_docs, k)
        ndcg = ndcg_at_k(retrieved_ids, relevant_docs, k)

        # Obtener tiempos
        search_time = response["search_time"]
        total_time = response["total_time"]

        # Guardar resultados de la consulta
        query_result = {
            "question": question,
            "relevant_docs": relevant_docs,
            "retrieved_docs": retrieved_ids,
            "answer": response["answer"],
            "sources": response["sources"],
            "precision_at_k": p_at_k,
            "recall_at_k": r_at_k,
            "ndcg_at_k": ndcg,
            "search_time": search_time,
            "total_time": total_time
        }

        results["queries"].append(query_result)
        results["metrics"]["precision_at_k"].append(p_at_k)
        results["metrics"]["recall_at_k"].append(r_at_k)
        results["metrics"]["ndcg_at_k"].append(ndcg)
        results["metrics"]["search_times"].append(search_time)
        results["metrics"]["total_times"].append(total_time)

    # Calcular promedios
    results["avg_metrics"] = {
        "avg_precision_at_k": np.mean(results["metrics"]["precision_at_k"]),
        "avg_recall_at_k": np.mean(results["metrics"]["recall_at_k"]),
        "avg_ndcg_at_k": np.mean(results["metrics"]["ndcg_at_k"]),
        "avg_search_time": np.mean(results["metrics"]["search_times"]),
        "avg_total_time": np.mean(results["metrics"]["total_times"])
    }

    return results

# Ejecutar evaluaciones
K = 5
print(f"🔄 Iniciando evaluación con k={K}...\\n")

qdrant_results = evaluate_rag_system(qdrant_rag, "Qdrant", test_queries, k=K)
weaviate_results = evaluate_rag_system(weaviate_rag, "Weaviate", test_queries, k=K)

print("✅ Evaluación completada")


🔄 Iniciando evaluación con k=5...\n


Evaluando Weaviate: 100%|██████████| 10/10 [00:36<00:00,  3.64s/it]

✅ Evaluación completada


## 10. Tabla Comparativa de Resultados


In [13]:
# Crear tabla comparativa
comparison_df = pd.DataFrame([
    {
        "Sistema": "Qdrant",
        "P@5": f"{qdrant_results['avg_metrics']['avg_precision_at_k']:.3f}",
        "R@5": f"{qdrant_results['avg_metrics']['avg_recall_at_k']:.3f}",
        "nDCG@5": f"{qdrant_results['avg_metrics']['avg_ndcg_at_k']:.3f}",
        "Latencia Búsqueda (ms)": f"{qdrant_results['avg_metrics']['avg_search_time']*1000:.1f}",
        "Tiempo Total (ms)": f"{qdrant_results['avg_metrics']['avg_total_time']*1000:.1f}"
    },
    {
        "Sistema": "Weaviate",
        "P@5": f"{weaviate_results['avg_metrics']['avg_precision_at_k']:.3f}",
        "R@5": f"{weaviate_results['avg_metrics']['avg_recall_at_k']:.3f}",
        "nDCG@5": f"{weaviate_results['avg_metrics']['avg_ndcg_at_k']:.3f}",
        "Latencia Búsqueda (ms)": f"{weaviate_results['avg_metrics']['avg_search_time']*1000:.1f}",
        "Tiempo Total (ms)": f"{weaviate_results['avg_metrics']['avg_total_time']*1000:.1f}"
    }
])

print("📊 TABLA COMPARATIVA - QDRANT VS WEAVIATE")
print("=" * 80)
print(comparison_df.to_string(index=False))
print("=" * 80)

# Mostrar tabla con Rich para mejor visualización
table = Table(title="Comparación Qdrant vs Weaviate")
table.add_column("Sistema", style="cyan")
table.add_column("P@5", justify="right")
table.add_column("R@5", justify="right")
table.add_column("nDCG@5", justify="right")
table.add_column("Latencia Búsqueda (ms)", justify="right")
table.add_column("Tiempo Total (ms)", justify="right")

for _, row in comparison_df.iterrows():
    table.add_row(
        row["Sistema"],
        row["P@5"],
        row["R@5"],
        row["nDCG@5"],
        row["Latencia Búsqueda (ms)"],
        row["Tiempo Total (ms)"]
    )

console.print(table)


📊 TABLA COMPARATIVA - QDRANT VS WEAVIATE
 Sistema   P@5   R@5 nDCG@5 Latencia Búsqueda (ms) Tiempo Total (ms)
  Qdrant 0.220 1.000  0.926                   28.0            5713.3
Weaviate 0.220 1.000  0.926                    9.0            3639.8


                          Comparación Qdrant vs Weaviate                          
┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Sistema  ┃   P@5 ┃   R@5 ┃ nDCG@5 ┃ Latencia Búsqueda (ms) ┃ Tiempo Total (ms) ┃
┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Qdrant   │ 0.220 │ 1.000 │  0.926 │                   28.0 │            5713.3 │
│ Weaviate │ 0.220 │ 1.000 │  0.926 │                    9.0 │            3639.8 │
└──────────┴───────┴───────┴────────┴────────────────────────┴───────────────────┘

## 11. Análisis Comparativo


In [14]:
# Análisis detallado
analysis = f"""
## ANÁLISIS COMPARATIVO: QDRANT VS WEAVIATE

### Métricas de Calidad:
- **Precision@5**: Qdrant ({qdrant_results['avg_metrics']['avg_precision_at_k']:.3f}) vs Weaviate ({weaviate_results['avg_metrics']['avg_precision_at_k']:.3f})
- **Recall@5**: Qdrant ({qdrant_results['avg_metrics']['avg_recall_at_k']:.3f}) vs Weaviate ({weaviate_results['avg_metrics']['avg_recall_at_k']:.3f})
- **nDCG@5**: Qdrant ({qdrant_results['avg_metrics']['avg_ndcg_at_k']:.3f}) vs Weaviate ({weaviate_results['avg_metrics']['avg_ndcg_at_k']:.3f})

### Rendimiento:
- **Latencia de Búsqueda**: Qdrant ({qdrant_results['avg_metrics']['avg_search_time']*1000:.1f}ms) vs Weaviate ({weaviate_results['avg_metrics']['avg_search_time']*1000:.1f}ms)
- **Tiempo Total**: Qdrant ({qdrant_results['avg_metrics']['avg_total_time']*1000:.1f}ms) vs Weaviate ({weaviate_results['avg_metrics']['avg_total_time']*1000:.1f}ms)

### Conclusiones:

**Calidad de Resultados:**
Ambos sistemas muestran rendimientos similares en términos de calidad, lo cual es esperado dado que:
- Utilizan los mismos embeddings (all-MiniLM-L6-v2)
- Procesan el mismo corpus de documentos
- Implementan búsqueda por similitud coseno

**Rendimiento y Latencia:**
{'Qdrant' if qdrant_results['avg_metrics']['avg_search_time'] < weaviate_results['avg_metrics']['avg_search_time'] else 'Weaviate'} muestra mejor rendimiento en búsqueda.
- Qdrant utiliza índices HNSW optimizados para consultas rápidas
- Weaviate (simulado) tiene overhead mínimo pero en implementación real sería diferente

**Facilidad de Uso:**
- **Qdrant**: API más simple, configuración directa, ideal para casos de uso específicos
- **Weaviate**: Esquemas más ricos, mejor manejo de metadatos complejos, GraphQL integrado

**Recomendación:**
- **Qdrant**: Mejor para aplicaciones que priorizan velocidad y simplicidad
- **Weaviate**: Mejor para aplicaciones que requieren esquemas complejos y consultas avanzadas
"""

print(analysis)



## ANÁLISIS COMPARATIVO: QDRANT VS WEAVIATE

### Métricas de Calidad:
- **Precision@5**: Qdrant (0.220) vs Weaviate (0.220)
- **Recall@5**: Qdrant (1.000) vs Weaviate (1.000)
- **nDCG@5**: Qdrant (0.926) vs Weaviate (0.926)

### Rendimiento:
- **Latencia de Búsqueda**: Qdrant (28.0ms) vs Weaviate (9.0ms)
- **Tiempo Total**: Qdrant (5713.3ms) vs Weaviate (3639.8ms)

### Conclusiones:

**Calidad de Resultados:**
Ambos sistemas muestran rendimientos similares en términos de calidad, lo cual es esperado dado que:
- Utilizan los mismos embeddings (all-MiniLM-L6-v2)
- Procesan el mismo corpus de documentos
- Implementan búsqueda por similitud coseno

**Rendimiento y Latencia:**
Weaviate muestra mejor rendimiento en búsqueda.
- Qdrant utiliza índices HNSW optimizados para consultas rápidas
- Weaviate (simulado) tiene overhead mínimo pero en implementación real sería diferente

**Facilidad de Uso:**
- **Qdrant**: API más simple, configuración directa, ideal para casos de uso específicos
- **W

## 12. Bonus: Filtrado por Metadatos y Consultas Híbridas


In [15]:
print("🎯 BONUS: Ejemplos de Filtrado por Metadatos y Consultas Híbridas\\n")

# Ejemplo 1: Filtro por categoría
print("=" * 60)
print("EJEMPLO 1: Búsqueda filtrada por categoría 'medicina'")
print("=" * 60)

question1 = "¿Qué aplicaciones tiene la tecnología en salud?"
filters1 = {"category": "medicina"}

# Qdrant
print("\\n🔹 QDRANT:")
qdrant_filtered = qdrant_rag.query(question1, k=3, filters=filters1)
print(f"Pregunta: {question1}")
print(f"Filtro: {filters1}")
print(f"Respuesta: {qdrant_filtered['answer']}")
print(f"Documentos encontrados: {[r['doc_id'] for r in qdrant_filtered['search_results']]}")
print(f"Fuentes: {qdrant_filtered['sources']}")

# Weaviate
print("\\n🔹 WEAVIATE:")
weaviate_filtered = weaviate_rag.query(question1, k=3, filters=filters1)
print(f"Pregunta: {question1}")
print(f"Filtro: {filters1}")
print(f"Respuesta: {weaviate_filtered['answer']}")
print(f"Documentos encontrados: {[r['doc_id'] for r in weaviate_filtered['search_results']]}")
print(f"Fuentes: {weaviate_filtered['sources']}")


🎯 BONUS: Ejemplos de Filtrado por Metadatos y Consultas Híbridas\n
EJEMPLO 1: Búsqueda filtrada por categoría 'medicina'
\n🔹 QDRANT:
Pregunta: ¿Qué aplicaciones tiene la tecnología en salud?
Filtro: {'category': 'medicina'}
Respuesta: La IA is applied in diagnostics for imagens, analogues of genomas and temprana de enfermedades. The algorithms of aprendizaje automático can identify patrons in radiografas, resonancias magnéticas and tomografas computarizadas.
Documentos encontrados: [17, 6, 1]
Fuentes: ['Revista Mente Abierta', 'Revista Salud Digital', 'Psicología Clínica']
\n🔹 WEAVIATE:
Pregunta: ¿Qué aplicaciones tiene la tecnología en salud?
Filtro: {'category': 'medicina'}
Respuesta: La IA is applied in diagnostics for imagens, analogues of genomas and temprana de enfermedades. The algorithms of aprendizaje automático can identify patrons in radiografas, resonancias magnéticas and tomografas computarizadas.
Documentos encontrados: [17, 6, 1]
Fuentes: ['Revista Mente Abierta', 'Revis

In [16]:
# Ejemplo 2: Filtro por rango de fechas
print("\\n" + "=" * 60)
print("EJEMPLO 2: Búsqueda filtrada por rango de fechas (2025)")
print("=" * 60)

question2 = "¿Cuáles son las tendencias tecnológicas recientes?"
filters2 = {"date": ("2025-01-01", "2025-12-31")}

# Qdrant
print("\\n🔹 QDRANT:")
qdrant_date_filtered = qdrant_rag.query(question2, k=5, filters=filters2)
print(f"Pregunta: {question2}")
print(f"Filtro: {filters2}")
print(f"Respuesta: {qdrant_date_filtered['answer']}")
print(f"Documentos encontrados: {[r['doc_id'] for r in qdrant_date_filtered['search_results']]}")
print(f"Fechas: {[docs[r['doc_id']-1]['date'] for r in qdrant_date_filtered['search_results']]}")
print(f"Fuentes: {qdrant_date_filtered['sources']}")

# Weaviate
print("\\n🔹 WEAVIATE:")
weaviate_date_filtered = weaviate_rag.query(question2, k=5, filters=filters2)
print(f"Pregunta: {question2}")
print(f"Filtro: {filters2}")
print(f"Respuesta: {weaviate_date_filtered['answer']}")
print(f"Documentos encontrados: {[r['doc_id'] for r in weaviate_date_filtered['search_results']]}")
print(f"Fechas: {[docs[r['doc_id']-1]['date'] for r in weaviate_date_filtered['search_results']]}")
print(f"Fuentes: {weaviate_date_filtered['sources']}")


\n============================================================
EJEMPLO 2: Búsqueda filtrada por rango de fechas (2025)
\n🔹 QDRANT:
Pregunta: ¿Cuáles son las tendencias tecnológicas recientes?
Filtro: {'date': ('2025-01-01', '2025-12-31')}
Respuesta: No encontré información relevante.
Documentos encontrados: []
Fechas: []
Fuentes: []
\n🔹 WEAVIATE:
Pregunta: ¿Cuáles son las tendencias tecnológicas recientes?
Filtro: {'date': ('2025-01-01', '2025-12-31')}
Respuesta: nTexto: Importance de los policadoresnTexto: Abejas, mariposas y aves ayudan en la reproducción de plantas y producción de alimentos. Sin policadores, muchos cultivos no podan
Documentos encontrados: [23, 8, 30, 20, 26]
Fechas: ['2025-07-17', '2025-06-01', '2025-08-01', '2025-05-01', '2025-03-10']
Fuentes: ['Tech Diario', 'Economía Sostenible', 'Aprendizaje Automático Hoy', 'Biodiversidad Global', 'Seguridad Digital Hoy']


In [17]:
# Ejemplo 3: Consulta híbrida (texto + múltiples filtros)
print("\\n" + "=" * 60)
print("EJEMPLO 3: Consulta híbrida (texto + categoría)")
print("=" * 60)

question3 = "¿Qué información hay sobre salud?"
filters3 = {"category": "salud"}  # Solo documentos de categoría salud

# Qdrant
print("\\n🔹 QDRANT:")
qdrant_hybrid = qdrant_rag.query(question3, k=3, filters=filters3)
print(f"Pregunta: {question3}")
print(f"Filtro: {filters3}")
print(f"Respuesta: {qdrant_hybrid['answer']}")
print(f"Documentos encontrados: {[r['doc_id'] for r in qdrant_hybrid['search_results']]}")
print(f"Categorías: {[r['category'] for r in qdrant_hybrid['search_results']]}")
print(f"Fuentes: {qdrant_hybrid['sources']}")

# Weaviate
print("\\n🔹 WEAVIATE:")
weaviate_hybrid = weaviate_rag.query(question3, k=3, filters=filters3)
print(f"Pregunta: {question3}")
print(f"Filtro: {filters3}")
print(f"Respuesta: {weaviate_hybrid['answer']}")
print(f"Documentos encontrados: {[r['doc_id'] for r in weaviate_hybrid['search_results']]}")
print(f"Categorías: {[r['category'] for r in weaviate_hybrid['search_results']]}")
print(f"Fuentes: {weaviate_hybrid['sources']}")


\n============================================================
EJEMPLO 3: Consulta híbrida (texto + categoría)
\n🔹 QDRANT:
Pregunta: ¿Qué información hay sobre salud?
Filtro: {'category': 'salud'}
Respuesta: Ttulo: Alimentos ricos en fibranTexto: Legumbres, frutas, cereales integrales y semillas favorecen la digestión y la salud intestinal. Una dieta rica en fibra puede prevenir enfermedades cardiovasculares.nnTtulo: Ventajas del ejercicio regularnTexto: Reduce el riesgo de enfermedades, mejora el estado de ánimo y fortalece
Documentos encontrados: [21, 27, 3]
Categorías: ['salud', 'salud', 'salud']
Fuentes: ['Salud Activa', 'Portal Nutrición y Vida', 'Guía Nutricional']
\n🔹 WEAVIATE:
Pregunta: ¿Qué información hay sobre salud?
Filtro: {'category': 'salud'}
Respuesta: Ttulo: Alimentos ricos en fibranTexto: Legumbres, frutas, cereales integrales y semillas favorecen la digestión y la salud intestinal. Una dieta rica en fibra puede prevenir enfermedades cardiovasculares.nnTtulo: Ventajas